In [3]:
import pandas as pd
import pyspark.sql.functions as F
from datetime import datetime
from pyspark.sql.types import *

import numpy as np
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("mode.chained_assignment", None)

In [4]:
super_srag = spark.read.parquet('gs://ai-covid19-datalake/standard/super-srag/super_srag_v1.parquet')

In [5]:
# creating a year attribute
super_srag = super_srag.withColumn('YEAR_SIN_PRI', F.year('DT_SIN_PRI'))

In [6]:
super_srag.limit(2).toPandas()

,NU_NOTIFIC,CS_SEXO,DT_NASC,AGE_AT_NOTIF,AGE_GROUP,CS_GESTANT,CS_RACA,CS_ETINIA,CS_ESCOL_N,SG_UF,CO_MUN_RES,SEM_PRI,SEM_NOT,DIST_PRI_NOTIFIC,DT_SIN_PRI,SG_UF_NOT,CO_MUN_NOT,SURTO_SG,NOSOCOMIAL,AVE_SUINO,VACINA,HOSPITAL,DIST_PRI_INTERNA,SUPORT_VEN,UTI,DIST_PRI_ENTUTI,CLASSI_OUT,CRITERIO,EVOLUCAO,CLASSI_FIN,SYMP_GROUP1,SYMP_GROUP2,SYMP_GROUP3,SYMP_GROUP4,OUTRO_SIN,OUTRO_DES,RF_GROUP1,RF_GROUP2,RF_GROUP3,RF_GROUP4,OBES_IMC,OUT_MORBI,MORB_DESC,RAIOX_RES,DIST_PRI_RAIOX,TOMO_RES,DIST_PRI_TOMO,AMOSTRA,TP_AMOSTRA,DT_COLETA,DIST_PRI_COLETA,PP_IF_RESUL,PP_TRA_RESUL,DIST_PRI_TRA,PP_PCR_RESUL,DIST_PRI_PCR,PP_RES_SOR_IGA,PP_RES_SOR_IGM,PP_RES_SOR_IGG,DIST_PRI_SOR,DIST_PRI_IF,DIST_PRI_NOTIFIC_Q,DIST_PRI_INTERNA_Q,DIST_PRI_ENTUTI_Q,DIST_PRI_SAIDUTI_Q,DIST_PRI_EVOLUCA_Q,DIST_PRI_ENCERRA_Q,DIST_PRI_RAIOX_Q,DIST_PRI_TOMO_Q,DIST_PRI_COLETA_Q,DIST_PRI_SOR_Q,DIST_PRI_PCR_Q,DIST_PRI_TRA_Q,DIST_PRI_IF_Q,EPI_WEEK_YEAR,GMR_TRANSIT_STATIONS_AVG,GMR_GROCERY_AND_PHARMACY_AVG,GMR_RETAIL_AND_RECREATION_AVG,GMR_WORKPLACES_PERCENT_AVG,GMR_RESIDENTIAL_PERCENT_AVG,GMR_PARKS_PERCENT_AVG,GMR_TRANSIT_STATIONS_1WEEK_BEFORE_AVG,GMR_GROCERY_AND_PHARMACY_1WEEK_BEFORE_AVG,GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_AVG,GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_AVG,GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_AVG,GMR_PARKS_PERCENT_1WEEK_BEFORE_AVG,GMR_TRANSIT_STATIONS_2WEEKS_AVG,GMR_GROCERY_AND_PHARMACY_2WEEKS_AVG,GMR_RETAIL_AND_RECREATION_2WEEKS_AVG,GMR_WORKPLACES_PERCENT_2WEEKS_AVG,GMR_RESIDENTIAL_PERCENT_2WEEKS_AVG,GMR_PARKS_PERCENT_2WEEKS_AVG,INMET_TEMP_C_AVG,INMET_RELATIVE_AIR_HUMIDITY_AVG,INMET_DAILY_PRECIPT_AVG,INMET_TEMP_C_1WEEK_BEFORE_AVG,INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_AVG,INMET_DAILY_PRECIPT_1WEEK_BEFORE_AVG,INMET_TEMP_C_2WEEKS_BEFORE_AVG,INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_AVG,INMET_DAILY_PRECIPT_2WEEKS_BEFORE_AVG,GMR_TRANSIT_STATIONS_Q,GMR_GROCERY_AND_PHARMACY_Q,GMR_RETAIL_AND_RECREATION_Q,GMR_WORKPLACES_PERCENT_Q,GMR_RESIDENTIAL_PERCENT_Q,GMR_PARKS_PERCENT_Q,GMR_TRANSIT_STATIONS_1WEEK_BEFORE_Q,GMR_GROCERY_AND_PHARMACY_1WEEK_BEFORE_Q,GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_Q,GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_Q,GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_Q,GMR_PARKS_PERCENT_1WEEK_BEFORE_Q,GMR_TRANSIT_STATIONS_2WEEKS_Q,GMR_GROCERY_AND_PHARMACY_2WEEKS_Q,GMR_RETAIL_AND_RECREATION_2WEEKS_Q,GMR_WORKPLACES_PERCENT_2WEEKS_Q,GMR_RESIDENTIAL_PERCENT_2WEEKS_Q,GMR_PARKS_PERCENT_2WEEKS_Q,INMET_TEMP_C_Q,INMET_RELATIVE_AIR_HUMIDITY_Q,INMET_DAILY_PRECIPT_Q,INMET_TEMP_C_1WEEK_BEFORE_Q,INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_Q,INMET_DAILY_PRECIPT_1WEEK_BEFORE_Q,INMET_TEMP_C_2WEEKS_BEFORE_Q,INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_Q,INMET_DAILY_PRECIPT_2WEEKS_BEFORE_Q,YEAR_SIN_PRI
0,68719768275,M,1967-01-25,54,7,6,4,None,NaN,BA,290240,19,20,7,2021-05-15,BA,291800,None,NaN,NaN,None,1,NaN,NaN,NaN,None,None,None,None,None,9,3,2,1,NaN,None,1,1,1,1,None,None,None,None,None,None,None,2,NaN,None,NaN,9,9,None,6,None,4,4,4,None,None,3,6,6,6,6,6,6,6,6,6,6,6,6,19-2021,-26.208333,31.788462,-24.079137,-0.099905,7.358333,-44.62533,-25.097436,34.628571,-20.930314,0.773764,6.81768,-44.472296,-31.233161,17.257009,-30.477193,-4.605239,8.130556,-46.526854,22.265185,71.299468,1.115789,23.258355,74.984254,2.16,24.092008,74.733094,1.653012,3,5,3,4,2,2,4,5,4,5,2,2,3,4,3,3,3,1,2,3,2,3,4,3,4,4,2,2021
1,68719899483,M,1959-10-20,61,8,6,4,None,9.0,BA,292050,19,20,8,2021-05-11,BA,292050,None,2.0,2.0,None,1,8.0,3.0,2.0,None,None,None,None,None,3,1,3,1,1.0,ASTENIA E INAPETENCIA,1,1,1,1,None,None,None,None,None,6,None,1,1.0,2021-05-17,6.0,9,9,None,7,None,4,4,4,None,None,3,4,6,6,6,6,6,6,4,6,6,6,6,19-2021,-26.208333,31.788462,-24.079137,-0.099905,7.358333,-44.62533,-25.097436,34.628571,-20.930314,0.773764,6.81768,-44.472296,-31.233161,17.257009,-30.477193,-4.605239,8.130556,-46.526854,22.265185,71.299468,1.115789,23.258355,74.984254,2.16,24.092008,74.733094,1.653012,3,5,3,4,2,2,4,5,4,5,2,2,3,4,3,3,3,1,2,3,2,3,4,3,4,4,2,2021


In [7]:
# criar PP_RES_SOR

In [8]:
cols = [
        'NU_NOTIFIC', 'CLASSI_FIN', 'CRITERIO', 'EVOLUCAO', 
        'AGE_AT_NOTIF', 'AGE_GROUP', 'DIST_PRI_NOTIFIC_Q', 'EPI_WEEK_YEAR', 'YEAR_SIN_PRI', 'SG_UF',
        'DIST_PRI_INTERNA_Q', 'DIST_PRI_ENTUTI_Q', 'DIST_PRI_SAIDUTI_Q', 'DIST_PRI_EVOLUCA_Q', 'DIST_PRI_EVOLUCA_Q', 'DIST_PRI_ENCERRA_Q',
        'SYMP_GROUP1', 'SYMP_GROUP2', 'SYMP_GROUP3', 'SYMP_GROUP4',
        'RF_GROUP1', 'RF_GROUP2', 'RF_GROUP3', 'RF_GROUP4', 
        'SUPORT_VEN', 'UTI', 'HOSPITAL',
        'DIST_PRI_RAIOX_Q', 'DIST_PRI_TOMO_Q', 'DIST_PRI_IF_Q', 'DIST_PRI_TRA_Q', 'DIST_PRI_PCR_Q', 'DIST_PRI_SOR_Q',
        'RAIOX_RES', 'TOMO_RES', 'PP_IF_RESUL', 'PP_TRA_RESUL', 'PP_PCR_RESUL', 'PP_RES_SOR_IGA', 'PP_RES_SOR_IGA', 'PP_RES_SOR_IGG',
        'GMR_TRANSIT_STATIONS_1WEEK_BEFORE_Q', 'GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_Q', 'GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_Q', 'GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_Q',
        'GMR_TRANSIT_STATIONS_2WEEKS_Q', 'GMR_RETAIL_AND_RECREATION_2WEEKS_Q', 'GMR_RESIDENTIAL_PERCENT_2WEEKS_Q', 'GMR_WORKPLACES_PERCENT_2WEEKS_Q',
        'INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_Q', 'INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_Q'
       ]

In [9]:
super_srag.select('CRITERIO').groupBy('CRITERIO').count().orderBy('CRITERIO').show()

+----------+-------+
|  CRITERIO|  count|
+----------+-------+
|      null| 268668|
|         1|1681220|
|         2|  21057|
|24/11/2020|      1|
|         3|  74441|
|         4|  65269|
+----------+-------+



In [10]:
super_srag.select(['CRITERIO', 'CLASSI_FIN']).groupBy(['CRITERIO', 'CLASSI_FIN']).count().orderBy(['CRITERIO', 'CLASSI_FIN']).show(50)

+----------+----------+-------+
|  CRITERIO|CLASSI_FIN|  count|
+----------+----------+-------+
|      null|      null| 200052|
|      null|         1|    219|
|      null|         2|    270|
|      null|         3|    340|
|      null|         4|  21758|
|      null|         5|  46029|
|         1|         1|   9524|
|         1|         2|  18105|
|         1|         3|   3897|
|         1|         4| 491126|
|         1|         5|1158568|
|         2|         1|     48|
|         2|         2|      8|
|         2|         3|     94|
|         2|         4|   8594|
|         2|         5|  12313|
|24/11/2020|         1|      1|
|         3|         1|    170|
|         3|         2|     31|
|         3|         3|    358|
|         3|         4|  47036|
|         3|         5|  26846|
|         4|         1|     41|
|         4|         2|      6|
|         4|         3|    229|
|         4|         4|   3396|
|         4|         5|  61597|
+----------+----------+-------+



In [11]:
# showing some piece of descriptive analysis for sorological
super_srag.select(['PP_RES_SOR_IGA', 'PP_RES_SOR_IGA', 'PP_RES_SOR_IGG']).groupBy(['PP_RES_SOR_IGA', 'PP_RES_SOR_IGA', 'PP_RES_SOR_IGG']).count().show()

+--------------+--------------+--------------+-------+
|PP_RES_SOR_IGA|PP_RES_SOR_IGA|PP_RES_SOR_IGG|  count|
+--------------+--------------+--------------+-------+
|             9|             9|             4|     34|
|             9|             9|             5|      5|
|             4|             4|             1|  88203|
|             4|             4|             2|  63411|
|             5|             5|             4|     31|
|             1|             1|             4|   1763|
|             5|             5|             1|     80|
|             5|             5|             5|     52|
|             1|             1|             1|    802|
|             2|             2|             3|      4|
|             9|             9|             2|    840|
|             4|             4|             4|1946930|
|             3|             3|             3|      7|
|             2|             2|             2|   1201|
|             9|             9|             3|      2|
|         

In [12]:
super_srag.select('PP_PCR_RESUL', 'YEAR_SIN_PRI', 'CLASSI_FIN').filter(F.col('PP_PCR_RESUL') == '2').groupBy(['YEAR_SIN_PRI', 'CLASSI_FIN']).count().orderBy(['YEAR_SIN_PRI', 'CLASSI_FIN']).show()

+------------+----------+------+
|YEAR_SIN_PRI|CLASSI_FIN| count|
+------------+----------+------+
|        2019|         5|     1|
|        2020|         5|487262|
|        2021|         5|365946|
+------------+----------+------+



In [13]:
super_srag.select('PP_IF_RESUL', 'YEAR_SIN_PRI', 'CLASSI_FIN').filter(F.col('PP_IF_RESUL') == '2').groupBy(['YEAR_SIN_PRI', 'CLASSI_FIN']).count().orderBy(['YEAR_SIN_PRI', 'CLASSI_FIN']).show()

+------------+----------+-----+
|YEAR_SIN_PRI|CLASSI_FIN|count|
+------------+----------+-----+
|        2020|      null|   21|
|        2020|         2|    2|
|        2020|         5| 5779|
|        2021|      null|  300|
|        2021|         1|    7|
|        2021|         2|    9|
|        2021|         3|    1|
|        2021|         5| 7814|
+------------+----------+-----+



In [14]:
super_srag.select('PP_TRA_RESUL', 'YEAR_SIN_PRI', 'CLASSI_FIN').filter(F.col('PP_TRA_RESUL') == '2').groupBy(['YEAR_SIN_PRI', 'CLASSI_FIN']).count().orderBy(['YEAR_SIN_PRI', 'CLASSI_FIN']).show()

+------------+----------+-----+
|YEAR_SIN_PRI|CLASSI_FIN|count|
+------------+----------+-----+
|        2020|      null|  126|
|        2020|         1|    4|
|        2020|         2|   10|
|        2020|         3|    4|
|        2020|         5|16885|
|        2021|      null| 2465|
|        2021|         1|   64|
|        2021|         2|   22|
|        2021|         3|    2|
|        2021|         5|77543|
+------------+----------+-----+



In [15]:
# not covid dataset 
super_srag.select(['CLASSI_FIN', 'YEAR_SIN_PRI']).filter((F.col('CLASSI_FIN') ==  '1') | (F.col('CLASSI_FIN') ==  '2')).groupBy('YEAR_SIN_PRI').count().orderBy('YEAR_SIN_PRI').show()

+------------+-----+
|YEAR_SIN_PRI|count|
+------------+-----+
|        2018|   21|
|        2019|16278|
|        2020| 7017|
|        2021| 5107|
+------------+-----+



<hr />
<hr />
<hr />

# Dataset 2: 
## Selecting for covid-19 confirmed cases only those with positive using LABORATORIAL criteria
### data balancing with stratified sampling strategy using the col: AGE_GROUP (with 0.05 of fraction for each age group)

In [16]:
count = super_srag.filter(F.col('CRITERIO') == '1').count()
super_srag.filter(F.col('PP_PCR_RESUL') == '1')\
          .select(['AGE_GROUP']).groupBy('AGE_GROUP').count().orderBy('AGE_GROUP')\
          .withColumn('%', (F.col('count')/count)*100).show()

+---------+-----+--------------------+
|AGE_GROUP|count|                   %|
+---------+-----+--------------------+
|        1|  791|  0.0470491666765801|
|        2| 1130|  0.0672130952522573|
|        3| 1017| 0.06049178572703156|
|        4|  729| 0.04336136852999607|
|        5|  914|0.054365282354480675|
|        6|  808| 0.04806033713612733|
|        7|  950| 0.05650658450411011|
|        8|  854| 0.05079644543843161|
|        9|  614| 0.03652109777423537|
|       10|  431|0.025636145180285744|
|       11|  135|0.008029883061110384|
|       12|   30|0.001784418458024...|
+---------+-----+--------------------+



In [17]:
fractions = {1 : 0.05, 2 : 0.05, 3 : 0.05, 4 : 0.05, 5 : 0.05, 6 : 0.05,
             7 : 0.05, 8 : 0.05, 9 : 0.05, 10 : 0.05, 11 : 0.05, 12 : 0.05}

df_2_covid = super_srag.filter(F.col('PP_PCR_RESUL') == '2').sampleBy('AGE_GROUP', fractions=fractions, seed=2021)

In [18]:
count = df_2_covid.filter(F.col('CRITERIO') == '1').count()
print(count)
df_2_covid.filter(F.col('CRITERIO') == '1')\
          .select(['AGE_GROUP']).groupBy('AGE_GROUP').count().orderBy('AGE_GROUP')\
          .withColumn('%', (F.col('count')/count)*100).show()

41047
+---------+-----+-------------------+
|AGE_GROUP|count|                  %|
+---------+-----+-------------------+
|        1|  183|0.44583038955343873|
|        2|  166| 0.4044144517260701|
|        3|  320| 0.7795941238092917|
|        4| 1372|  3.342509805832339|
|        5| 3889|  9.474504835919799|
|        6| 6080| 14.812288352376543|
|        7| 7946| 19.358296586839476|
|        8| 8881|  21.63617316734475|
|        9| 6961|    16.958608424489|
|       10| 4116| 10.027529417497014|
|       11| 1083|  2.638438862767072|
|       12|   50|0.12181158184520186|
+---------+-----+-------------------+



In [19]:
df_2_not_covid = super_srag.filter((F.col('CLASSI_FIN') ==  '1') | (F.col('CLASSI_FIN') ==  '2')).withColumn('CLASSI_FIN', F.lit('1'))

In [20]:
df_2_not_covid.count()

28423

In [21]:
df_2 = df_2_not_covid.union(df_2_covid)

In [22]:
df_2.select('CLASSI_FIN').groupBy('CLASSI_FIN').count().show()

+----------+-----+
|CLASSI_FIN|count|
+----------+-----+
|         5|42484|
|         1|28423|
+----------+-----+



In [23]:
df_2 = df_2.select(cols).persist()

<hr />
<hr />
<hr />

## Preprocessing Dataset 1
### some description before the script

In [24]:
for col in cols: 
    if col != "NU_NOTIFIC":
        
        print("==== " + col + " ====")
        
        df_2.select(col).printSchema()
        
        print("GroupBy: all")
        
        count = df_2.count()
        df_2.select([col]).groupBy(col).count().orderBy(col)\
            .withColumn('%', (F.col('count')/count)*100).show()
        
        print("GroupBy: covid")
        
        count = df_2.filter((F.col('CRITERIO') == '1') & (F.col('CLASSI_FIN') == '5')).count()
        df_2.filter(F.col('CLASSI_FIN') == '5').select([col]).groupBy(col).count().orderBy(col)\
                                               .withColumn('%', (F.col('count')/count)*100).show()
        
        print("GroupBy: not covid")
        count = count = df_2.filter(F.col('CLASSI_FIN') == '1').count()
        df_2.filter(F.col('CLASSI_FIN') == '1').select([col]).groupBy(col).count().orderBy(col)\
                                               .withColumn('%', (F.col('count')/count)*100).show()
        
        print('=========================')
        print('=========================')

==== CLASSI_FIN ====
root
 |-- CLASSI_FIN: string (nullable = true)

GroupBy: all
+----------+-----+-----------------+
|CLASSI_FIN|count|                %|
+----------+-----+-----------------+
|         1|28423|40.13867706038525|
|         5|42389|59.86132293961476|
+----------+-----+-----------------+

GroupBy: covid
+----------+-----+------------------+
|CLASSI_FIN|count|                 %|
+----------+-----+------------------+
|         5|42389|103.37267716919476|
+----------+-----+------------------+

GroupBy: not covid
+----------+-----+-----+
|CLASSI_FIN|count|    %|
+----------+-----+-----+
|         1|28423|100.0|
+----------+-----+-----+

==== CRITERIO ====
root
 |-- CRITERIO: string (nullable = true)

GroupBy: all
+----------+-----+--------------------+
|  CRITERIO|count|                   %|
+----------+-----+--------------------+
|      null| 1568|     2.2143139580862|
|         1|68635|    96.9256623171214|
|         2|  131| 0.18499689318194656|
|24/11/2020|    1|0.001412

### Casting variables

In [25]:
cols_to_cast = ['RAIOX_RES', 'UTI', 'SUPORT_VEN', 'YEAR_SIN_PRI', 'AGE_GROUP']

In [26]:
for col in cols_to_cast:
    df_2 = df_2.withColumn(col, F.col(col).cast('string'))

### Filling nulls

In [27]:
# null to "9"
cols_null_to_9 = ['TOMO_RES', 'RAIOX_RES', 'HOSPITAL', 'UTI', 'SUPORT_VEN', 'EVOLUCAO', 'CRITERIO']
for col in cols_null_to_9:
    df_2 = df_2.withColumn(col, F.when(F.col(col).isNull(), '9').otherwise(F.col(col)))

In [28]:
# null to string

In [29]:
cols_null_to_string = ['SG_UF']
for col in cols_null_to_string:
    df_2 = df_2.withColumn(col, F.when(F.col(col).isNull(), 'NULL').otherwise(F.col(col)))

### Taking out invalid values

In [30]:
df_2 = df_2.filter(F.col('TOMO_RES') != '20/11/2020')
df_2 = df_2.filter(F.col('HOSPITAL') != 'RJ')

### Showing final descriptive

In [31]:
for col in cols: 
    if col != "NU_NOTIFIC":
        
        print("==== " + col + " ====")
        
        df_2.select(col).printSchema()
        
        print("GroupBy: all")
        
        count = df_2.count()
        df_2.select([col]).groupBy(col).count().orderBy(col)\
            .withColumn('%', (F.col('count')/count)*100).show()
        
        print("GroupBy: covid")
        
        count = df_2.filter(F.col('CLASSI_FIN') == '5').count()
        df_2.filter(F.col('CLASSI_FIN') == '5').select([col]).groupBy(col).count().orderBy(col)\
                                               .withColumn('%', (F.col('count')/count)*100).show()
        
        print("GroupBy: not covid")
        count = count = df_2.filter(F.col('CLASSI_FIN') == '1').count()
        df_2.filter(F.col('CLASSI_FIN') == '1').select([col]).groupBy(col).count().orderBy(col)\
                                               .withColumn('%', (F.col('count')/count)*100).show()
        
        print('=========================')
        print('=========================')

==== CLASSI_FIN ====
root
 |-- CLASSI_FIN: string (nullable = true)

GroupBy: all
+----------+-----+------------------+
|CLASSI_FIN|count|                 %|
+----------+-----+------------------+
|         1|28422| 40.13783169281609|
|         5|42389|59.862168307183914|
+----------+-----+------------------+

GroupBy: covid
+----------+-----+-----+
|CLASSI_FIN|count|    %|
+----------+-----+-----+
|         5|42389|100.0|
+----------+-----+-----+

GroupBy: not covid
+----------+-----+-----+
|CLASSI_FIN|count|    %|
+----------+-----+-----+
|         1|28422|100.0|
+----------+-----+-----+

==== CRITERIO ====
root
 |-- CRITERIO: string (nullable = true)

GroupBy: all
+--------+-----+-------------------+
|CRITERIO|count|                  %|
+--------+-----+-------------------+
|       1|68635|  96.92703111098558|
|       2|  131|0.18499950572651142|
|       3|  355| 0.5013345384191722|
|       4|  122|0.17228961602010986|
|       9| 1568|  2.214345228848625|
+--------+-----+-------------

<hr />
<hr />
<hr />

In [ ]:
df = {}
# starting on 135 to complete the earlier execution
df_keys = range(0,500)

for df_key in df_keys:
    
    # establishing the proportion for each category on EPI_WEEK_YEAR attribute
    epi_weeks = super_srag.select('EPI_WEEK_YEAR').distinct().rdd.flatMap(lambda x: x).collect()

    fractions = {}

    for e_w in epi_weeks:
        fractions[e_w] = 0.05
    
    # getting stratified sample for super srag by AGE_GROUP when PCR_RESULT is positive for covid
    df_covid = super_srag.filter((F.col('CRITERIO') == '1') & (F.col('CLASSI_FIN') == '5'))\
                         .sampleBy('EPI_WEEK_YEAR', fractions=fractions, seed=df_key)
    
    # getting all NOT COVID records from super srag
    df_not_covid = super_srag.filter((F.col('CLASSI_FIN') ==  '1') | (F.col('CLASSI_FIN') ==  '2'))\
                             .withColumn('CLASSI_FIN', F.lit('1'))
    
    # unioning the datasets and putting into a sample dataset to be written and submitted to the model
    df[df_key] = df_not_covid.union(df_covid)

    
    # writing the sample dataset
    pad_key = str(df_key).zfill(3)
    id_dataset = 'laboratorialpositivecovid'
    
    df[df_key].write.parquet('gs://ai-covid19-datalake/trusted/XGB-StratifiedSampleDatasets-4/dataset_4_'+id_dataset+'_'+pad_key+'.parquet', mode='overwrite')
#     df[df_key].coalesce(1).write.csv('gs://ai-covid19-datalake/trusted/XGB-StratifiedSampleDatasets-2/dataset_2_'+id_dataset+'_'+pad_key+'.csv', header='true', mode='overwrite')
    print('{} done'.format(df_key))

0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
110 done
111 done
112 done
113 done
114 done
115 done
116 done
117 done
118 done
119 done
120 done
121 done
122 done
123

In [ ]:
print('finished')

finished
